# **Documentação - Carga Tabelas Agronegocio**

### 📄 Descrição do processo:
Este processo tem como objetivo carregar tabelas do agronegocio disponiveis no site do govBR e os dados completos de importação e exportação do ComexStat para tabelas Delta em ambiente Spark (Databricks).

---

### 📚 Bibliotecas Importadas:
- `concurrent.futures.ThreadPoolExecutor`: Para execução paralela do download das tabelas auxiliares.
- `requests`: Para fazer requisições HTTP às URLs das tabelas.
- `pandas`: Para leitura dos arquivos CSV das tabelas auxiliares.
- `zipfile`, `io`: Para manipulação de arquivos ZIP e leitura em buffer.
- `os`: Para manipulação de arquivos locais temporários.

---

### 📦 Origem dos Dados:
- Todas as tabelas são obtidas do site oficial da Balança Comercial (https://balanca.economia.gov.br).
- São divididas em dois grandes blocos:
  - **Tabelas auxiliares** (como NCM, SH, CUCI, CGCE, UF, etc).
  - **Arquivos completos de importação e exportação** (em `.zip`, contendo grandes volumes de dados históricos).

---

### ⚙️ Processamento:

- `CMD 2 - Bibliotecas`: Importa todas as bibliotecas necessárias para execução dos scripts.

- `CMD 4 - Criação do Banco`:  
  Criação do banco de dados Delta `bd_becomex` para armazenamento das tabelas.

- `CMD 4 (cont.) - Dicionário de Tabelas Auxiliares`:  
  Dicionário com URLs e nomes das tabelas auxiliares para automatizar o processo de importação.

- `CMD 5 - Função de Importação COMEXSTAT`:  
  Função `import_full_export_import` que faz:
  - Download do arquivo ZIP da base de importação/exportação.
  - Extração e leitura do CSV.
  - Escrita como tabela Delta com inferência automática de schema.

- `CMD 6 - Funções de Importação de Tabelas Auxiliares`:  
  - `baixar_csv`: Baixa e lê um CSV da URL.
  - `importar_table_assistants`: Converte cada CSV em DataFrame Spark e salva como tabela Delta.

- `CMD 7 - Execução do Processo`:  
  Executa:
  - Importação de todas as tabelas auxiliares.
  - Importação completa dos dados de **importação** (`IMP_COMPLETA.zip`).
  - Importação completa dos dados de **exportação** (`EXP_COMPLETA.zip`).

---

### 🛬 Tabelas Geradas:

No banco `bd_becomex` são salvas as seguintes tabelas Delta:

#### Tabelas Auxiliares:
- `ncm`
- `sh`
- `ncm_cuci`
- `ncm_isic`
- `ncm_cgce`
- `ncm_fat_agreg`
- `ncm_ppe`
- `ncm_ppi`
- `ncm_unidade`
- `nbm_ncm`
- `nbm`
- `estados`
- `via`
- `urf`
- `paises`
- `blocos`
- `municipios`

#### Tabelas ComexStat:
- `import`
- `export`

---

### 🧨 Problemas Comuns:

#### _Erro de conexão com a URL_
- Verificar se a URL está ativa e disponível.
- Tentar rodar novamente em outro horário.

#### _Schema divergente entre execuções_
- Pode haver mudança na estrutura de dados do site.
- Verificar nomes de colunas e aplicar cast ou tratamento específico.

#### _Ambiente Databricks não detectado_
- A função tenta copiar arquivos para o DBFS, se estiver fora do Databricks, ele mantém os arquivos localmente.

---

### 📩 Observações Finais:
- As tabelas geradas servem de apoio para análise da Balança Comercial.
- As tabelas Delta permitem leitura rápida e otimização futura com ZORDER e OPTIMIZE, se necessário.
- A execução paralela nas auxiliares agiliza o processo.
- É recomendável agendar essa carga periodicamente para manter os dados atualizados.

---

> _"A simplicidade é o último grau de sofisticação."_ – Leonardo da Vinci  


In [0]:
from concurrent.futures import ThreadPoolExecutor
import requests
import pandas as pd
import zipfile
import io
from io import StringIO
import os

In [0]:

spark.sql("CREATE DATABASE IF NOT EXISTS bd_becomex LOCATION '/mnt/dnc/'")

In [0]:
dic_tabelas = {
    'tabela_ncm': {
        'nome': 'ncm',
        'url': 'https://balanca.economia.gov.br/balanca/bd/tabelas/NCM.csv'
    },
    'tabela_sh': {
        'nome': 'sh',
        'url': 'https://balanca.economia.gov.br/balanca/bd/tabelas/NCM_SH.csv'
    },
    'tabela_cuci': {
        'nome': 'ncm_cuci',
        'url': 'https://balanca.economia.gov.br/balanca/bd/tabelas/NCM_CUCI.csv'
    },
    'tabela_isic': {
        'nome': 'ncm_isic',
        'url': 'https://balanca.economia.gov.br/balanca/bd/tabelas/NCM_ISIC.csv'
    },
    'tabela_isic_cuci': {
        'nome': 'isic_cuci',
        'url': 'https://balanca.economia.gov.br/balanca/bd/tabelas/ISIC_CUCI.csv' #apagar
    },
    'tabela_cgce': {
        'nome': 'ncm_cgce',
        'url': 'https://balanca.economia.gov.br/balanca/bd/tabelas/NCM_CGCE.csv'
    },
    'tabela_fator_agregado': {
        'nome': 'ncm_fat_agreg',
        'url': 'https://balanca.economia.gov.br/balanca/bd/tabelas/NCM_FAT_AGREG.csv'
    },
    'tabela_ppe': {
        'nome': 'ncm_ppe',
        'url': 'https://balanca.economia.gov.br/balanca/bd/tabelas/NCM_PPE.csv'
    },
    'tabela_ppi': {
        'nome': 'ncm_ppi',
        'url': 'https://balanca.economia.gov.br/balanca/bd/tabelas/NCM_PPI.csv'
    },
    'tabela_unidade': {
        'nome': 'ncm_unidade',
        'url': 'https://balanca.economia.gov.br/balanca/bd/tabelas/NCM_UNIDADE.csv'
    },
    'tabela_nbm_ncm': {
        'nome': 'nbm_ncm',
        'url': 'https://balanca.economia.gov.br/balanca/bd/tabelas/NBM_NCM.csv'
    },
    'tabela_nbm': {
        'nome': 'nbm',
        'url': 'https://balanca.economia.gov.br/balanca/bd/tabelas/NBM.csv'
    },
    'tabela_uf': {
        'nome': 'estados',
        'url': 'https://balanca.economia.gov.br/balanca/bd/tabelas/UF.csv'
    },
    'tabela_via': {
        'nome': 'via',
        'url': 'https://balanca.economia.gov.br/balanca/bd/tabelas/VIA.csv'
    },
    'tabela_urf': {
        'nome': 'urf',
        'url': 'https://balanca.economia.gov.br/balanca/bd/tabelas/URF.csv'
    },
    'tabela_pais': {
        'nome': 'paises',
        'url': 'https://balanca.economia.gov.br/balanca/bd/tabelas/PAIS.csv'
    },
    'tabela_blocos': {
        'nome': 'blocos',
        'url': 'https://balanca.economia.gov.br/balanca/bd/tabelas/PAIS_BLOCO.csv'
    },
    'tabela_municipios': {
        'nome': 'municipios',
        'url': 'https://balanca.economia.gov.br/balanca/bd/tabelas/UF_MUN.csv'
    }
}


In [0]:
def import_full_export_import(url, caminho_delta, nome_tabela):
    try:
        # Requisição para obter o arquivo ZIP
        response = requests.get(url)
        response.raise_for_status()

        # Abrir o conteúdo como ZIP
        with zipfile.ZipFile(io.BytesIO(response.content)) as z:
            print("Arquivos no zip:", z.namelist())

            # Abrir o primeiro arquivo dentro do ZIP
            with z.open(z.namelist()[0]) as csv_file:
                # Caminhos temporários para o DBFS e local
                local_temp_path = "/tmp/temp_file.csv"  # Caminho local na máquina do driver
                dbfs_temp_path = "dbfs:/tmp/temp_file.csv"

                # Escreve temporariamente no driver local
                with open(local_temp_path, 'wb') as f:
                    f.write(csv_file.read())

                # Se for no Databricks, mova o arquivo para o DBFS
                if 'dbutils' in globals():  # Verifica se está no Databricks
                    try:
                        dbutils.fs.cp(f"file:{local_temp_path}", dbfs_temp_path)
                    except Exception as e:
                        print(f"❌ Erro ao mover o arquivo para DBFS: {e}")
                else:
                    print("Ambiente local detectado, mantendo o arquivo local.")

                # Ler com Spark diretamente do DBFS ou local dependendo do ambiente
                path_to_read = dbfs_temp_path if 'dbutils' in globals() else local_temp_path
                df_spark = spark.read.csv(path_to_read, header=True, sep=';', inferSchema=True, encoding='latin1')

                # Salva como tabela Delta
                df_spark.write.format("delta") \
                    .option("overwriteSchema", True) \
                    .mode("overwrite") \
                    .saveAsTable(f"{caminho_delta}.{nome_tabela}")

                print(f"✅ Tabela {nome_tabela} salva com sucesso em: {caminho_delta}.{nome_tabela}")

                # Remove os arquivos temporários
                os.remove(local_temp_path)
                if 'dbutils' in globals():
                    try:
                        dbutils.fs.rm(dbfs_temp_path)
                    except Exception as e:
                        print(f"Erro ao remover o arquivo do DBFS: {e}")

    except requests.exceptions.RequestException as e:
        print(f"❌ Erro ao acessar a URL: {e}")
    except zipfile.BadZipFile as e:
        print(f"❌ Erro ao abrir o arquivo ZIP: {e}")
    except Exception as e:
        print(f"❌ Erro ao processar o arquivo CSV: {e}")


In [0]:
def baixar_csv(info):
    url = info['url']
    nome = info['nome']
    
    response = requests.get(url)
    response.raise_for_status()
    
    df = pd.read_csv(StringIO(response.text), sep=';')
    return nome, df

def import_table_assistants(dic_tabelas, nome_banco):

    with ThreadPoolExecutor() as executor:
        resultados = executor.map(baixar_csv, dic_tabelas.values())

        for nome, df in resultados:
            df_spark = spark.createDataFrame(df)
            tabela = f"{nome_banco}.{nome}"
            try: 
                df_spark.write.format("delta").option('overwriteSchema', True).mode("overwrite").saveAsTable(tabela)
                print(f"✅ {nome} salva como tabela: {tabela}")
            except:
                print(f"❌ {nome} salva como tabela: {tabela}")

In [0]:
import_table_assistants(dic_tabelas, "bd_becomex")
import_full_export_import('https://balanca.economia.gov.br/balanca/bd/comexstat-bd/ncm/IMP_COMPLETA.zip', 'bd_becomex', 'import') # importação
import_full_export_import('https://balanca.economia.gov.br/balanca/bd/comexstat-bd/ncm/EXP_COMPLETA.zip', 'bd_becomex', 'export') # exportação

In [0]:
%sql
select * from export limit 10

In [0]:
%sql
SELECT 
    n.no_ncm_por, 
    e.*
FROM  
    ncm n 
left JOIN 
    export e ON n.co_ncm = e.co_ncm
WHERE 
    n.no_ncm_por = 'Cocaína e seus sais'


In [0]:
%sql
select co_ncm, no_ncm_por from ncm
order by co_ncm